<center>
    <h1> 
        Teste de Memória do Pré-Processamento
    </h1>
</center>
    <div style="text-align: right;"><h3>
        Carlos Eduardo Cassimiro da Silva
    </h3></div>

<center>
    <h4>Neste notebook, iremos começar o pré-processamento do conjunto de dados original, pois até então os testes estavam sendo feitos com uma amostra do original. Também verificaremos o quanto podemos reduzir de espaço o conjuto de dados original transformando as imagens em histogramas.</h4>
    Link do dataset: https://github.com/ricardobnjunior/Brazilian-Identity-Document-Dataset
</center>

<h4>Desafio #1: Classificação de documentos (RG, CNH e CPF) </h4>

 - Contextualização: Inúmeras áreas de diferentes organizações (usualmente como parte de um processo de um backoffice) recepcionam documentos dos seus clientes para formação de kits de documentação. Tais kits são, por exemplo, compartilhados com outros stakeholders das empresas. Conforme pode-se pressupor, um desafio nesse cenário refere-se ao fato de que o cliente pode enviar documentos, porém sem necessariamente indicar a qual tipo se o documento se refere (RG, CNH ou CPF, por exemplo). Dessa forma, ao invés de demandar um trabalho manual para essa leitura e classificação dos documentos, podemos construir um modelo de aprendizado que tenha capacidade de ler um conjunto de documentos (em .jpg, por exemplo) e, subsequentemente, realizar a classificação em três tipos distintos: RG, CNH e CPF.
 - Dataset: Para esse desafio utilizaremos um dataset público de RG, CNH e CPF (incluindo as imagens). Este repositório apresenta o conjunto de dados denominado Brazilian Identity Document Dataset (BID Dataset), o primeiro conjunto de dados público de documentos de identificação brasileiros. <br>

<h4>Roteiro</h4>
Módulos utilizados <br>
1. Abrindo todas as amostras de um tipo de documento <br>
1.1 Somatório do espaço ocupado <br>
2. Transformando as amostras em histogramas <br>
2.1 Somatório do espaço ocupado <br>
3. Economia <br>
4. Conclusão

##### Módulos utilizados

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import copy
from sklearn.preprocessing import MinMaxScaler
import os
from sys import getsizeof

# 1. Abrindo todas as amostras de um tipo de documento

In [2]:
path = 'imagens_originais/BID_Dataset/CPF_Frente/' # caminho do diretório
a = os.listdir(path) # os.listdir lista todos os arquivos ou pastas dentro do referido diretório 
doc_list = [] # lista para guardar as amostras
for i in range(2,len(a),3): # Laço para pegar os documentos. Como queremos só as imagens dos documentos,
    doc_list.append(cv2.imread(path+a[i]))                                  # pulamos os outros arquivos

## 1.1 Somatório do espaço ocupado

Para examinar o valor do espaço ocupado em memória, precisamos utilizar o 'getsizeof' para descobrir o valor ocupado por cada variável e multiplicar pelo valor de cada lista que as comportam. Como se trata de imagem, precisamos executar do nível mais baixo da estrutura de dados até o mais alto, como espaço de cada pixel, número de pixel, numero de linhas e multiplar pelo número e espaço ocupado por cada.

In [74]:
soma = getsizeof(doc_list[0][0][0][0])*len(doc_list[0][0][0])
soma += getsizeof(doc_list[0][0][0])
soma *= len(doc_list[0][0][0])
soma += getsizeof(doc_list[0][0])

aux = 0
for i in range(len(doc_list)):
    aux += len(doc_list[i][0])

soma *= aux
soma += (getsizeof(doc_list[i])*len(doc_list[i]))

soma += getsizeof(doc_list)
soma /= (1024*1024*1024)
'Espaço ocupado: '+ str(soma) + ' GB'

'Espaço ocupado: 1.763122363947332 GB'

# 2. Transformando as amostras em histogramas

In [75]:
#ksize = (70,70)
#cv2.cvtColor(), cv2.COLOR_BGR2RGB
#cv2.blur(img_aux, ksize)

doc = copy.deepcopy(doc_list)

for i in range(len(doc_list)):
    hist = [] 
    
    hist.append(cv2.calcHist(doc[i],[0],None,[256],[0,256]))  # Processo análogo ao anterior para 
    hist.append(cv2.calcHist(doc[i],[1],None,[256],[0,256]))  # cálcular as média dos histogramas
    hist.append(cv2.calcHist(doc[i],[2],None,[256],[0,256]))
    for j in range(0,3):
        mm = MinMaxScaler()
        hist[j] = mm.fit_transform(hist[j], hist[j])
    doc[i] = hist

## 2.1 Somatório do espaço ocupado

In [77]:
soma = getsizeof(doc[0][0][0])*len(doc[0][0])
soma += getsizeof(doc[0][0])*3
soma += getsizeof(doc[0])
soma *= len(doc)
soma += getsizeof(doc)
soma /= (1024*1024)
'Espaço ocupado: '+ str(soma) + ' MB'

'Espaço ocupado: 103.51992797851562 MB'

# 3. Economia

In [80]:
print('Economia de' ,100-(100*(103.51992797851562/1024)/1.763122363947332), '%')

Economia de 94.26621306985254 %


# 4. Conclusão

Podemos ver que, adatodando a estratégia de utilizar os histogramas ao invés das imagens originais, conseguimos reduzir em 94% o espaço ocupado do CPF_Frente, mas acredito que podemos generalizar esse valor para os outros documentos também. Reduzindo o valor nessa proporção, acredito que podemos utilizar todo o dataset sem problemas de memória.